In [ ]:
import os
import random
import cv2
import numpy as np
from glob import glob
from PIL import Image, ImageEnhance
import json

In [ ]:
TABLE_IMAGE_PATH = 'path/to/your/table_background.jpg'
CARD_IMAGES_FOLDER = 'path/to/your/cards_folder/'
OUTPUT_IMAGES_FOLDER = 'output/images/'
OUTPUT_LABELS_FOLDER = 'output/labels/'

PASTE_AREA = {
    'x_min': 0.1,
    'x_max': 0.9,
    'y_min': 0.1,
    'y_max': 0.9,
}

IMAGE_SIZE = (1280, 720)
MAX_TRIES = 50  

os.makedirs(OUTPUT_IMAGES_FOLDER, exist_ok=True)
os.makedirs(OUTPUT_LABELS_FOLDER, exist_ok=True)

In [ ]:
def load_card_images():
    paths = glob(os.path.join(CARD_IMAGES_FOLDER, '*.png'))
    return paths

In [ ]:
def random_transform(card_img):
    # Random rotation
    angle = random.uniform(-25, 25)
    card_img = card_img.rotate(angle, expand=True)

    # Random brightness
    enhancer = ImageEnhance.Brightness(card_img)
    card_img = enhancer.enhance(random.uniform(0.7, 1.3))

    return card_img

In [ ]:
def get_random_position(card_size, table_size):
    for _ in range(MAX_TRIES):
        x_min_area = int(PASTE_AREA['x_min'] * table_size[0])
        x_max_area = int(PASTE_AREA['x_max'] * table_size[0])
        y_min_area = int(PASTE_AREA['y_min'] * table_size[1])
        y_max_area = int(PASTE_AREA['y_max'] * table_size[1])

        x = random.randint(x_min_area, x_max_area - card_size[0])
        y = random.randint(y_min_area, y_max_area - card_size[1])
        return x, y
    return None, None  # If cannot find a valid spot

In [ ]:
def paste_card(base_img, card_img, position):
    base_img.paste(card_img, position, card_img.convert('RGBA'))

In [ ]:
def create_one_image(image_id, table_image, card_paths):
    table = table_image.copy()
    h_table, w_table = table.size
    annotations = []

    selected_cards = random.sample(card_paths, random.randint(4, 8))

    for card_path in selected_cards:
        card_img = Image.open(card_path).convert('RGBA')
        card_img = random_transform(card_img)

        card_w, card_h = card_img.size
        x, y = get_random_position((card_w, card_h), (w_table, h_table))

        if x is None:
            continue  # Skip if no position found

        paste_card(table, card_img, (x, y))

        card_name = os.path.splitext(os.path.basename(card_path))[0]

        annotations.append({
            'class': card_name,
            'bbox': [x, y, x + card_w, y + card_h]
        })

    output_img_path = os.path.join(OUTPUT_IMAGES_FOLDER, f'image_{image_id}.png')
    output_label_path = os.path.join(OUTPUT_LABELS_FOLDER, f'image_{image_id}.json')

    table.save(output_img_path)

    with open(output_label_path, 'w') as f:
        json.dump(annotations, f, indent=2)

In [ ]:
table_image = Image.open(TABLE_IMAGE_PATH).convert('RGB').resize(IMAGE_SIZE)
card_paths = load_card_images()

num_images = 100  # how many synthetic images you want
for i in range(num_images):
    create_one_image(i, table_image, card_paths)